In [ ]:
import twitter
import json
import operator
import tweepy
from prettytable import PrettyTable
from collections import Counter
from IPython.display import Image, display

In [ ]:
##Lectura de claves de Twitter que estan definidos en un archivo JSON
##sin trackeo de git

with open('twitter_keys.json') as data_file:    
    data = json.load(data_file)

OAUTH_TOKEN = data["OAUTH_TOKEN"]
OAUTH_TOKEN_SECRET = data["OAUTH_TOKEN_SECRET"]
CONSUMER_KEY = data["CONSUMER_KEY"]
CONSUMER_SECRET = data["CONSUMER_SECRET"]

In [ ]:
auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

In [ ]:
#Se realiza la consulta a Twitter
ARG_WOE_ID = 23424747
arg_trends = twitter_api.trends.place(_id=ARG_WOE_ID)


# Dumpear los tweets a disco

In [ ]:
#Dumpear a disco
with open("tweets-json-pto1.json", "w") as outfile:
    json.dump(arg_trends, outfile, indent=5)

In [ ]:
#Obtener de disco
with open('tweets-json-pto1.json', 'r') as f:
     tweets_arg = json.load(f)

### a. Los tres trends más RT del momento.

In [ ]:
x = filter(lambda x: x['tweet_volume'] is not None, tweets_arg[0]['trends'])
x = sorted(x, key=lambda x: x['tweet_volume'], reverse=True)
three_trends = x[:3]
print(json.dumps(three_trends, indent=1))

#### Se obtienen 300 tweets con las tendencias seleccionadas anteriormente

In [ ]:
result_1 = twitter_api.search.tweets(q=three_trends[0]['name'] , count=100)
result_2 = twitter_api.search.tweets(q=three_trends[1]['name'] , count=100)
result_3 = twitter_api.search.tweets(q=three_trends[2]['name'] , count=100)

## OBVIAR - SOLO ES PARA GUARDAR LOS DATOS Y EVITAR LAS MULTIPLES PETICIONES

In [ ]:
#Dumpear a disco
with open("tweets-trend1.json", "w") as outfile:
    json.dump(result_1, outfile, indent=5)

with open("tweets-trend2.json", "w") as outfile:
    json.dump(result_2, outfile, indent=5)

with open("tweets-trend3.json", "w") as outfile:
    json.dump(result_3, outfile, indent=5)
    
with open('tweets-trend1.json', 'r') as f:
     result_1 = json.load(f)

with open('tweets-trend2.json', 'r') as f:
     result_2 = json.load(f)

with open('tweets-trend3.json', 'r') as f:
     result_3 = json.load(f)

In [ ]:
# Se juntan todos los tweets en una sola lista
totales = list(result_1['statuses'] + result_2['statuses'] + result_3['statuses'])

### b. Listar nombres de usuarios que publicaron con los hashtag de los trends del punto anterior.

In [ ]:
screen_name = map( lambda i: totales[i]['user']['screen_name'], range(len(totales)))
list(screen_name)

### c. Cuales son las cinco palabras más utilizadas en los tweets del primer item.

In [ ]:
text = map( lambda i: totales[i]['text'], range(len(totales)))

In [ ]:
words = [ word for twt in text for word in twt.split() ]
c = Counter(words)
#print c.most_common()

pt = PrettyTable(field_names=['Words', 'Count'])
list(map(lambda r: pt.add_row(r), c.most_common()[:5]))
print(pt)

### d. Listar los primeros 10 usuarios con mayor cantidad de seguidores

In [ ]:
followers_count = list(map( lambda i: totales[i]['user']['followers_count'], range(len(totales))))
users = list(map( lambda i: totales[i]['user']['screen_name'], range(len(totales))))
data = dict(zip(users, followers_count))
data = sorted(data.items(), key=operator.itemgetter(1), reverse=True)

table = PrettyTable(field_names=["User", "Followers"])
list(map(lambda x: table.add_row(x), data[:10]))
print(table)

### e. Listar la ubicación (o en su defecto time-zone) del tweet

In [ ]:
time_zones = map( lambda i: totales[i]['user']['time_zone'], range(len(totales)))
time_zones = filter(lambda x:x is not None, time_zones)
list(time_zones)

### f. Listar los cinco tweets más populares.
Consideramos que un tweet popular es el que mas retweets tiene

In [ ]:
tweet = map( lambda i: totales[i], range(len(totales)))
tweet = sorted(tweet, key=lambda x: x['retweet_count'], reverse=True)[:5]
tweet = [ x['id'] for x in tweet]

favorite = list(map( lambda i: totales[i]['retweet_count'], range(len(totales))))
favorite = sorted(favorite, reverse=True)[:5]

In [ ]:
table = PrettyTable()
table.add_column('Id_Tweet', tweet)
table.add_column('Retweet_Count', favorite)
print(table)

### g. Listar a los seguidores del autor del tweet m ́as popular

In [ ]:
tweet = map( lambda i: totales[i], range(len(totales)))
tweet = sorted(tweet, key=lambda x: x['retweet_count'], reverse=True)[0]
id_user = tweet['user']['id']

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
users = api.followers(id_user)

In [ ]:
for user in users:
    print("Name: " + user.name)
    display(Image(url=user.profile_image_url))
    print("-------------------------------------")